# Import necessary dependencies

In [9]:
import pandas as pd
import numpy as np
import text_normalizer as tn
import warnings

warnings.filterwarnings("ignore")

# Load and normalize data

In [2]:
dataset = pd.read_csv(r'movie_reviews.csv')

# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# normalize datasets
norm_train_reviews = tn.normalize_corpus(train_reviews)
norm_test_reviews = tn.normalize_corpus(test_reviews)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


# Extract features from positive and negative reviews

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# consolidate all normalized reviews
norm_reviews = norm_train_reviews+norm_test_reviews
# get tf-idf features for only positive reviews
positive_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'positive']
ptvf = TfidfVectorizer(use_idf=True, min_df=0.05, max_df=0.95, ngram_range=(1,1), sublinear_tf=True)
ptvf_features = ptvf.fit_transform(positive_reviews)
# get tf-idf features for only negative reviews
negative_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'negative']
ntvf = TfidfVectorizer(use_idf=True, min_df=0.05, max_df=0.95, ngram_range=(1,1), sublinear_tf=True)
ntvf_features = ntvf.fit_transform(negative_reviews)
# view feature set dimensions
print(ptvf_features.shape, ntvf_features.shape)

(25000, 331) (25000, 331)


# Topic Modeling on Reviews

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import NMF
import topic_model_utils as tmu

pyLDAvis.enable_notebook()
total_topics = 10

## Display and visualize topics for positive reviews

In [5]:
# build topic model on positive sentiment review features
pos_nmf = NMF(n_components=total_topics, 
          random_state=42, alpha=0.1, l1_ratio=0.2)
pos_nmf.fit(ptvf_features)      
# extract features and component weights
pos_feature_names = ptvf.get_feature_names()
pos_weights = pos_nmf.components_
# extract and display topics and their components
pos_topics = tmu.get_topics_terms_weights(pos_weights, pos_feature_names)
tmu.print_topics_udf(topics=pos_topics,
                 total_topics=total_topics,
                 num_terms=15,
                 display_weights=False)

Topic #1 without weights
['like', 'not', 'think', 'really', 'say', 'would', 'get', 'know', 'thing', 'much', 'bad', 'go', 'lot', 'could', 'even']

Topic #2 without weights
['movie', 'see', 'watch', 'great', 'good', 'one', 'not', 'time', 'ever', 'enjoy', 'recommend', 'make', 'acting', 'like', 'first']

Topic #3 without weights
['show', 'episode', 'series', 'tv', 'watch', 'dvd', 'first', 'see', 'time', 'one', 'good', 'year', 'remember', 'ever', 'would']

Topic #4 without weights
['performance', 'role', 'play', 'actor', 'cast', 'good', 'well', 'great', 'character', 'excellent', 'give', 'also', 'support', 'star', 'job']

Topic #5 without weights
['man', 'young', 'old', 'two', 'get', 'year', 'woman', 'take', 'go', 'come', 'find', 'back', 'girl', 'father', 'friend']

Topic #6 without weights
['film', 'see', 'one', 'scene', 'make', 'not', 'time', 'director', 'horror', 'music', 'many', 'cinema', 'release', 'work', 'use']

Topic #7 without weights
['story', 'tell', 'character', 'true', 'book', '

In [10]:
pyLDAvis.sklearn.prepare(pos_nmf, ptvf_features, ptvf, R=15)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      14.357252        1       1  0.014664  0.005799
4      13.233860        1       2  0.056482  0.065233
5      12.933760        1       3  0.040293 -0.060175
1      11.316494        1       4 -0.222410 -0.076387
8      10.693920        1       5  0.078358  0.151408
3      10.532171        1       6  0.151155 -0.101375
6       7.560320        1       7  0.253692  0.048936
2       7.418161        1       8 -0.101996 -0.057984
9       6.489072        1       9 -0.205223  0.225435
7       5.464990        1      10 -0.065015 -0.200889, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
179   Default  2065.000000          love  2065.000000  15.0000  15.0000
279   Default  1945.000000         story  1945.000000  14.0000  14.0000
195   Default  2565.000000         movie  2565.000000  13.0000  13.0000
117   Default  1355.000000         funny  1355.000000  12.0000  12.0000
55    Default  1055.000000        comedy  1055.000000  11.0000  11.0000
107   Default  1895.000000          film  1895.000000  10.0000  10.0000
162   Default   825.000000         laugh   825.000000   9.0000   9.0000
263   Default  1175.000000          show  1175.000000   8.0000   8.0000
85    Default   681.000000       episode   681.000000   7.0000   7.0000
257   Default   669.000000        series   669.000000   6.0000   6.0000
169   Default   768.000000          life   768.000000   5.0000   5.0000
217   Default   680.000000   performance   680.000000   4.0000   4.0000
245   Default   619.000000          role   619.000000   3.0000   3.0000
223   Default   798.000000          play   798.000000   2.0000   2.0000
286   Default   652.000000          tell   652.000000   1.0000   1.0000
186    Topic1   163.964151         maybe   211.102332   1.6882  -4.8415
228    Topic1   242.612464        pretty   317.111517   1.6731  -4.4497
26     Topic1   309.015145           bad   419.511014   1.6352  -4.2077
247    Topic1   446.859755           say   643.700289   1.5759  -3.8389
19     Topic1   145.014644      anything   227.330473   1.4913  -4.9643
237    Topic1   522.341314        really   828.464707   1.4797  -3.6828
288    Topic1   337.092842         thing   543.542192   1.4632  -4.1208
187    Topic1   156.417072          mean   253.055665   1.4598  -4.8886
289    Topic1   523.237842         think   850.034573   1.4557  -3.6811
270    Topic1   229.367808     something   393.079793   1.4022  -4.5058
7      Topic1   198.654126      actually   342.681974   1.3957  -4.6496
79     Topic1   105.638211          else   183.898452   1.3866  -5.2811
283    Topic1   159.592233          sure   301.191519   1.3058  -4.8685
178    Topic1   276.957480           lot   523.368386   1.3045  -4.3173
124    Topic1   197.261326           guy   379.271749   1.2872  -4.6566
...       ...          ...           ...          ...      ...      ...
99     Topic9   133.490184        family   488.645440   1.4374  -4.2530
40     Topic9    88.167362           buy   323.259890   1.4358  -4.6678
14     Topic9   135.841950        always   519.108391   1.3944  -4.2355
18     Topic9   109.917708        anyone   428.757251   1.3739  -4.4473
114    Topic9   106.632712        friend   447.365588   1.3011  -4.4776
326    Topic9   153.541222         would   933.558639   0.9300  -4.1130
313    Topic9   152.050154         watch  1292.108164   0.5952  -4.1228
123    Topic9   149.202780         great  1383.292652   0.5081  -4.1417
117   Topic10  1355.897103         funny  1355.897103   2.9068  -1.7630
55    Topic10   980.942418        comedy  1055.312106   2.8337  -2.0867
162   Topic10   765.627217         laugh   825.811834   2.8311  -2.3345
146   Topic10   254.685511         humor   387.746145   2.4865  -3.4352
116   Topic10   235.869802           fun   546.606942   2.0664  -3.51

## Display and visualize topics for negative reviews

In [7]:
# build topic model on negative sentiment review features
neg_nmf = NMF(n_components=10, 
          random_state=42, alpha=0.1, l1_ratio=0.2)
neg_nmf.fit(ntvf_features)      
# extract features and component weights
neg_feature_names = ntvf.get_feature_names()
neg_weights = neg_nmf.components_
# extract and display topics and their components
neg_topics = tmu.get_topics_terms_weights(neg_weights, neg_feature_names)
tmu.print_topics_udf(topics=neg_topics,
                 total_topics=total_topics,
                 num_terms=15,
                 display_weights=False) 

Topic #1 without weights
['get', 'go', 'kill', 'guy', 'scene', 'take', 'end', 'back', 'start', 'around', 'look', 'one', 'thing', 'come', 'first']

Topic #2 without weights
['bad', 'movie', 'ever', 'acting', 'see', 'terrible', 'one', 'plot', 'effect', 'awful', 'not', 'even', 'make', 'horrible', 'special']

Topic #3 without weights
['film', 'make', 'not', 'see', 'would', 'director', 'one', 'many', 'may', 'bad', 'however', 'horror', 'no', 'say', 'feel']

Topic #4 without weights
['character', 'story', 'book', 'plot', 'main', 'seem', 'no', 'interesting', 'not', 'movie', 'read', 'end', 'feel', 'nothing', 'original']

Topic #5 without weights
['movie', 'think', 'would', 'not', 'like', 'say', 'watch', 'could', 'see', 'really', 'people', 'good', 'know', 'want', 'make']

Topic #6 without weights
['funny', 'comedy', 'laugh', 'joke', 'try', 'not', 'stupid', 'suppose', 'moment', 'fun', 'even', 'black', 'guy', 'character', 'really']

Topic #7 without weights
['actor', 'play', 'good', 'cast', 'role'

In [11]:
pyLDAvis.sklearn.prepare(neg_nmf, ntvf_features, ntvf, R=15)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      15.159817        1       1  0.076605 -0.021691
4      13.708854        1       2 -0.060157 -0.026393
2      11.157697        1       3  0.001907 -0.021786
3      10.813738        1       4  0.076284  0.049930
1      10.635604        1       5 -0.231589  0.020158
6      10.582598        1       6  0.107173  0.039257
7       8.862276        1       7  0.218109 -0.142682
9       7.120793        1       8 -0.242084 -0.125559
8       6.419647        1       9  0.053506 -0.048141
5       5.538976        1      10  0.000245  0.276907, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
107   Default  2236.000000       film  2236.000000  15.0000  15.0000
260   Default  1356.000000       show  1356.000000  14.0000  14.0000
117   Default  1240.000000      funny  1240.000000  13.0000  13.0000
28    Default  1776.000000        bad  1776.000000  12.0000  12.0000
310   Default  1111.000000      waste  1111.000000  11.0000  11.0000
55    Default   893.000000     comedy   893.000000  10.0000  10.0000
289   Default  1399.000000       time  1399.000000   9.0000   9.0000
189   Default  2357.000000      movie  2357.000000   8.0000   8.0000
157   Default   799.000000      laugh   799.000000   7.0000   7.0000
49    Default  1084.000000  character  1084.000000   6.0000   6.0000
176   Default   791.000000        man   791.000000   5.0000   5.0000
275   Default   930.000000      story   930.000000   4.0000   4.0000
148   Default   642.000000       joke   642.000000   3.0000   3.0000
187   Default   667.000000      money   667.000000   2.0000   2.0000
317   Default   639.000000      woman   639.000000   1.0000   1.0000
151    Topic1   296.182894       kill   318.826975   1.8129  -4.3284
130    Topic1   162.230095       head   221.157364   1.5767  -4.9303
67     Topic1   144.792485       dead   201.323472   1.5569  -5.0441
140    Topic1   139.108803      house   200.283826   1.5220  -5.0841
243    Topic1   192.881583        run   279.680171   1.5150  -4.7573
20     Topic1   218.199939     around   324.167125   1.4907  -4.6339
257    Topic1   151.465034      shoot   228.512367   1.4753  -4.9990
125    Topic1   295.348838        guy   483.584801   1.3935  -4.3312
272    Topic1   224.884183      start   382.346878   1.3558  -4.6038
65     Topic1   106.941458        cut   182.422547   1.3525  -5.3471
72     Topic1   125.686888        die   222.527374   1.3153  -5.1856
308    Topic1   102.504916       walk   188.514620   1.2773  -5.3894
106    Topic1   111.576802      fight   209.403931   1.2570  -5.3046
108    Topic1    87.375183    finally   165.255309   1.2492  -5.5491
68     Topic1   114.935979      death   217.874857   1.2470  -5.2750
...       ...          ...        ...          ...      ...      ...
304    Topic9    59.923876    version   294.737118   1.1528  -5.0670
303    Topic9    93.310733        use   480.903906   1.1061  -4.6241
167    Topic9    63.154657       live   327.356586   1.1003  -5.0145
325    Topic9    58.315564     writer   305.496884   1.0897  -5.0942
211    Topic9   121.985745     people   743.555845   0.9383  -4.3562
311    Topic9   154.980332      watch  1342.737374   0.5866  -4.1168
164    Topic9   119.715163       like  1298.007227   0.3623  -4.3750
201    Topic9   127.169026        not  2356.114616  -0.1734  -4.3146
177    Topic9    82.434764       many   631.931797   0.7090  -4.7481
175    Topic9    98.653405       make  1477.664778   0.0392  -4.5685
117   Topic10  1240.046669      funny  1240.046669   2.8934  -1.8896
55    Topic10   854.231692     comedy   893.748710   2.8481  -2.2623
148   Topic10   575.233228       joke   642.217885   2.7832  -2.6577
157   Topic10   647.703989      laugh   799.954468   2.6822  -2.5391
278   Topic10   122.392615    suppose   364.124516   1.8031  -4.20